In [1]:
import pandas as pd
import numpy as np
from subprocess import call
import glob

from glob import glob
import itertools

import matplotlib.pyplot as plt
import os
%load_ext autoreload
%autoreload 2
import re
import sys
import shutil
%matplotlib inline
# import phy
# import Python3_OpenEphys_con2dat_align_v2 as c2d
# import Python3_new_OpenEphys_orig as orig
import Binary as bn


In [2]:
def applyCAR(data, pr=0): #this input data should be in the form (channels,samples) ie (384,7400)
    
    if pr==1:
        print('Shape before: %s' %str(data.shape))

    ch_median = np.median(data,axis=1) #subtract median across each channel
    data = data-ch_median[:,None]
    time_median = np.median(data, axis=0) #subtract median across each time sample
    data = (data.T-time_median[:,None]).T
    
    if pr==1:
        print('Shape after: %s' %str(data.shape))
    
    return data

---

# Define the folder location first

Change the following before running:
1. folder_name

Double check that these are correct:
1. in_path
2. out_path

In [3]:
in_path = r"D:\mz_Data\RECORDING_DATA\HDAC_data"
out_path = r"D:\mz_Data\RECORDING_DATA\HDAC_data\SORTED_DATA"

folder_name = input("Copy the folder name here: ")    #make sure to change this before running!!

working_path = in_path + "\\" + folder_name
print('\nFile name: ' + folder_name)


folders = [os.path.join(working_path,fld) for fld in os.listdir(working_path) if 'experiment' in fld]

print('\n# of experiments: %d' %len(folders))

Copy the folder name here: pre_et306_cc033657_2022-03-15_11-40-00

File name: pre_et306_cc033657_2022-03-15_11-40-00

# of experiments: 1


---

# Concatenate all of the recordings to a single .dat file

### What happens?
Essentially in the cell above it created a new dictionary -- "Data" -- that is structured as follows: {Process: {Experiment: {recording} } } where the recording is split into each trial and the data associated to it

ie. {'100': {'0': {9: memmap(the_data)} } }

In [4]:
Unit='int16'
ChannelMap=[]

for Folder in folders:

    #File locations for use on windows
    Files = sorted(glob(Folder + r'\**\Neuropix-PXI-100.0\continuous.dat', recursive=True))
    InfoFiles = sorted(glob(Folder + r'\*\structure.oebin'))
    ttlFiles=sorted(glob(Folder + r'\**\Neuropix-PXI-100.0\TTL_1\timestamps.npy', recursive=True))
    tssFiles=sorted(glob(Folder + r'\**\Neuropix-PXI-100.0\timestamps.npy', recursive=True))
    
    sorted_index=[f.split('\\')[-5][9:] for f in sorted(ttlFiles)]
    ttl=[np.load(ttl)[0] for ttl in sorted(ttlFiles)]
    tss=[np.load(tss) for tss in sorted(tssFiles)]
    tmp=np.array(list(itertools.zip_longest(*tss, fillvalue=np.nan))).T
    sliceat=np.where((tmp-np.array(ttl)[:,None])==0)[1]
    
    Data, Rate = {}, {}
    for F,File in enumerate(Files):
        Exp, Rec, _, Proc = File.split('\\')[-5:-1]
        Exp = str(int(Exp[10:])-1)
        Rec = str(int(Rec[9:])-1)
        Proc = Proc.split('.')[0].split('-')[-1]
        if Proc not in Data.keys(): Data[Proc], Rate[Proc] = {}, {}
        if '_' in Proc: Proc = Proc.split('_')[0]

#         print('Loading recording', int(Rec)+1, '...')

        if Exp not in Data[Proc]: Data[Proc][Exp] = {}
        Data[Proc][Exp][Rec] = np.memmap(File, dtype='int16', mode='c')
        
        Info = bn.literal_eval(open(InfoFiles[F]).read())
        ProcIndex = [Info['continuous'].index(_) for _ in Info['continuous']
                     if str(_['recorded_processor_id']) == Proc][0]

        ChNo = Info['continuous'][ProcIndex]['num_channels']
        if Data[Proc][Exp][Rec].shape[0]%ChNo:
            print('Rec', Rec, 'is broken')
            del(Data[Proc][Exp][Rec])
            continue

        SamplesPerCh = Data[Proc][Exp][Rec].shape[0]//ChNo
        Data[Proc][Exp][Rec] = Data[Proc][Exp][Rec].reshape((SamplesPerCh, ChNo))
        Rate[Proc][Exp] = Info['continuous'][ProcIndex]['sample_rate']

    for Proc in Data.keys():
        for Exp in Data[Proc].keys():
            if Unit.lower() in ['uv', 'mv']:
                ChInfo = Info['continuous'][ProcIndex]['channels']
                Data[Proc][Exp] = bn.BitsToVolts(Data[Proc][Exp], ChInfo, Unit)

            if ChannelMap: Data[Proc][Exp] = bn.ApplyChannelMap(Data[Proc][Exp], ChannelMap)

    print('Done.')

Done.


### This is to check the length of each recording in seconds
you'll find that the first 5 (from 0-4) are really short, which is due to how the TTL signal starts each session

In [5]:
Data['100']['0']={int(k):v for k,v in Data['100']['0'].items()}
for k,v in sorted(Data['100']['0'].items()):
    print(k,v.shape[0]/30000)


0 0.05243333333333333
1 0.06826666666666667
2 0.034133333333333335
3 0.06826666666666667
4 0.0896
5 3.0108
6 3.0104
7 2.99
8 3.0104
9 3.0004
10 3.0208
11 2.9904
12 2.98
13 3.0104
14 3.0104
15 3.0104
16 3.0004
17 2.98
18 3.0004
19 3.0004
20 3.0103666666666666
21 3.0208
22 2.9994
23 2.99
24 3.0104
25 2.99
26 3.0104
27 2.98
28 3.0108
29 3.0084
30 3.0004
31 3.010466666666667
32 3.0108
33 3.0104


### This is to check shape of each recording data frame
It creates a new dictionary -- "Data_align" -- and shows that each trial has 384 channels recorded

In [6]:
Data_align=dict()
for i,rec in enumerate(sorted_index):
    Data_align[rec]=Data['100']['0'][int(rec)-1][sliceat[i]:]


Data_align={int(k):v for k,v in Data_align.items()}
for k,v in sorted(Data_align.items()):
    print(k,v.shape)

1 (1414, 384)
2 (1588, 384)
3 (910, 384)
4 (1256, 384)
5 (2101, 384)
6 (89990, 384)
7 (89920, 384)
8 (89608, 384)
9 (89927, 384)
10 (89420, 384)
11 (90029, 384)
12 (89531, 384)
13 (89336, 384)
14 (90048, 384)
15 (89742, 384)
16 (90028, 384)
17 (89737, 384)
18 (89247, 384)
19 (89652, 384)
20 (89946, 384)
21 (90048, 384)
22 (90046, 384)
23 (89833, 384)
24 (89549, 384)
25 (89847, 384)
26 (89551, 384)
27 (89846, 384)
28 (89342, 384)
29 (89959, 384)
30 (90000, 384)
31 (89562, 384)
32 (89857, 384)
33 (89918, 384)
34 (90028, 384)


### Correctly pad each recording (NOT the first 5 short ones, tho)

First it applies the ___Common Average Referencing___, then does the rest

It's correctly aligning them by having each trial be an identical length

This is essentially doing two things
1. If any trial is longer than the 3 sec length of recording, only the first 3*30000 samples will be used
2. If any trial is shorter than the 3 sec length of recording, it will be 0 padded to the 3*30000 samples


**-----Important-----**

If you get an error similar to "MemoryError: Unable to allocate 38.6 GiB for an array with shape (13500000, 384) and data type float64", this means the jupyter notebook has reached it's maximum memory working with the data.

You need to Restart the Kernel and clear all output to fix this!

In [7]:
length_recording = 3
sample_rate = 30000

trial_samples = length_recording*sample_rate

print('length of each recording: %d seconds' %length_recording)
print('target samples per recording: %d' %trial_samples)



data_arr=[]
for k,v in sorted(Data_align.items())[5:]: # this excludes the first 5 TTL signal recordings
    
    to_CAR = np.array(v).T      #reshape to be the correct type for applyCAR function
    post_CAR = applyCAR(to_CAR) #applies the CAR function here
    post_CAR = post_CAR.T       #reshape back to fit the rest of the code
    
    if post_CAR.shape[0]>=trial_samples:
        data_arr.append(post_CAR[:trial_samples,:])
    elif post_CAR.shape[0]<trial_samples:
        data_arr.append(np.pad(post_CAR,((0,trial_samples-post_CAR.shape[0]),(0,0)), mode='constant', constant_values=0))

# for i in data_arr: # this can be used to check that each trial is the correct length and shape
#     print(i.shape)

data_todat=np.vstack(data_arr)
print("Final concat shape: {0} -- should be (samp,ch)".format(data_todat.shape))

# Depending on how many trials you have, this may take a while to finish running

length of each recording: 3 seconds
target samples per recording: 90000
Final concat shape: (2610000, 384) -- should be (samp,ch)


---

# Double check and make sure everything worked correctly

In [8]:
print('# of trials expected: %d' %(len(Data_align)-5))
print('# of trials at the end: %d' %(data_todat.shape[0]/(trial_samples)))

print('\n# of ch expected: %d' %(v.shape[1]))
print('# of ch at the end: %d' %(data_todat.shape[1]))

print('\n# of samples/trial expected: %d' %(trial_samples))
print('# of samples/trial at the end: %d' %(data_todat.shape[0]/(len(Data_align)-5)))


if (
    len(Data_align)-5 != data_todat.shape[0]/(trial_samples) or 
    v.shape[1] != data_todat.shape[1] or 
    trial_samples != data_todat.shape[0]/(len(Data_align)-5)
    ):
    raise TypeError("Something went wrong, check the data again...")

# of trials expected: 29
# of trials at the end: 29

# of ch expected: 384
# of ch at the end: 384

# of samples/trial expected: 90000
# of samples/trial at the end: 90000


---

# Finally, save the new file to a specific location
This will create a folder in the specific sorted_folder you want

In [9]:
output_path = out_path + '\\' + folder_name
print(output_path)

os.makedirs(output_path) #this makes a new folder with the same file name as above

ff = os.path.join(output_path,'openephys.dat')
with open(ff,'ab') as fi:
    data_todat.astype('int16').tofile(fi)       #This also makes sure that it is "int16" data type
    print('DONE!')
    

D:\mz_Data\RECORDING_DATA\HDAC_data\SORTED_DATA\pre_et306_cc033657_2022-03-15_11-40-00
DONE!


---

# You're done running the notebook for the recording!!!

---

---

---